In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import random
from datetime import datetime
import pandas
import wandb

In [ ]:
wandb.login()

In [ ]:
train_input=None
train_output=None
valid_input=None
valid_output=None
test_input=None
test_output=None

In [ ]:
def loadData():
    global train_input, train_output, valid_input, valid_output, test_input, test_output
    data_train = pandas.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv', header=None)
    train_input = data_train.iloc[:,0]
    train_output = data_train.iloc[:,1]
    data_valid = pandas.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_valid.csv', header=None)
    valid_input = data_valid.iloc[:,0]
    valid_output = data_valid.iloc[:,1]
    data_test = pandas.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_test.csv', header=None)
    test_input = data_test.iloc[:,0]
    test_output = data_test.iloc[:,1]
    max_inp_len=0
    max_out_len=0
    for i in train_input:
        max_inp_len=max(max_inp_len,len(i))
    for i in train_output:
        max_out_len=max(max_out_len,len(i))
    for i in valid_input:
        max_inp_len=max(max_inp_len,len(i))
    for i in valid_output:
        max_out_len=max(max_out_len,len(i))
    for i in test_input:
        max_inp_len=max(max_inp_len,len(i))
    for i in test_output:
        max_out_len=max(max_out_len,len(i))
    return max_inp_len+1,max_out_len+1

In [ ]:
SOW_token = 0
EOW_token = 1
PAD_token = 2

In [ ]:
class Dictionary:
    def __init__(self, max_inp_length):
        self.index = {}
        self.char = {0: "#", 1: "$",2:"*"}
        self.n_chars = 3  # Count SOS and EOS
        self.max_inp_len=max_inp_len
        self.max_out_len=max_out_len

    def preprocess(self, w):
        for x in w:
            for c in x:
                if c not in self.index:
                    self.index[c]=self.n_chars
                    self.char[self.n_chars]=c
                    self.n_chars+=1
                
    def createBatches(self, words, batch_size):
        x=[]
        for w in words:
            a=[]
            for i in w:
                a.append(self.index[i])
            a.append(EOW_token)
            #padd here
            diff=self.max_inp_len-len(a)
            a.extend([PAD_token]*diff)
            a_tensor=torch.tensor(a, dtype=torch.long,device=device)
            x.append(a_tensor)
        batches=[]
        size=len(x)
        for i in range(0,size,batch_size):
            if i+batch_size>=size:
                break
            temp=torch.stack(x[i:i+batch_size]).to(device)
            batches.append(temp.transpose(0,1))
        return batches
    
    def driver(self, words,batch_size):
        self.preprocess(words)
        return self.createBatches(words,batch_size)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, params):
        super(Encoder, self).__init__()
        self.hs = params["hs"]
        self.dp = nn.Dropout(params["de"])
        self.nl = params["nl"]
        self.bs = params["bs"]
        self.es = params["es"]
        self.ct = params["ct"]
        self.em = nn.Embedding(input_size, params["es"])
        self.bi = params["bi"]
        if(params["ct"] == "GRU"):
            self.gru = nn.GRU(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])
        elif(params["ct"] == "LSTM"):
            self.lstm = nn.LSTM(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])
        elif(params["ct"] == "RNN"):
            self.rnn = nn.RNN(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])

    def forward(self, input, hidden):
        output = self.dp(self.em(input).view(-1,self.bs, self.es))
        
        if(self.ct == "RNN"):
            _, hidden = self.rnn(output, hidden)

        if(self.ct == "LSTM"):
            _, (hidden, cell) = self.lstm(output)
            
        if(self.ct == "GRU"):
            _, hidden = self.gru(output, hidden)
            
        if self.bi:
            hidden = hidden.reshape(2, hidden.size(0)//2, hidden.size(1), hidden.size(2))
            hidden = torch.add(hidden[0]*0.5, hidden[1]*0.5)
            
            if(self.ct == "LSTM"):
                cell = cell.reshape(2, cell.size(0)//2, cell.size(1), cell.size(2))
                cell = torch.add(cell[0]*0.5, cell[1]*0.5)
        
        if self.ct != "LSTM":
            return hidden
        else:
            return hidden,cell

    def initHidden(self):
        if self.bi==False:
            return torch.zeros(self.nl, self.bs, self.hs, device=device)
        else:
            return torch.zeros(2*self.nl, self.bs, self.hs, device=device)

In [ ]:
class EncoderAttention(nn.Module):
    def __init__(self, input_size, params):
        super(EncoderAttention, self).__init__()
        self.hs = params["hs"]
        self.dp = nn.Dropout(params["de"])
        self.nl = params["nl"]
        self.bs = params["bs"]
        self.es = params["es"]
        self.ct = params["ct"]
        self.em = nn.Embedding(input_size, params["es"])
        self.bi = params["bi"]
        if(params["ct"] == "GRU"):
            self.gru = nn.GRU(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])
        elif(params["ct"] == "LSTM"):
            self.lstm = nn.LSTM(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])
        elif(params["ct"] == "RNN"):
            self.rnn = nn.RNN(params["es"], params["hs"], params["nl"], dropout = params["de"], bidirectional = params["bi"])

    def forward(self, input, hidden):
        output = self.dp(self.em(input).view(-1,self.bs, self.es))
        
        if(self.ct == "RNN"):
            output, hidden = self.rnn(output, hidden)

        if(self.ct == "LSTM"):
            output, (hidden, cell) = self.lstm(output)
            
        if(self.ct == "GRU"):
            output, hidden = self.gru(output, hidden)
            
        if self.bi:
            hidden = hidden.reshape(2, hidden.size(0)//2, hidden.size(1), hidden.size(2))
            hidden = torch.add(hidden[0]*0.5, hidden[1]*0.5)
            
            if(self.ct == "LSTM"):
                cell = cell.reshape(2, cell.size(0)//2, cell.size(1), cell.size(2))
                cell = torch.add(cell[0]*0.5, cell[1]*0.5)
            
            output = output.permute(2, 1, 0)
            output = torch.split(output, output.shape[0]//2)
            output = torch.add(output[0].permute(2, 1, 0)*0.5, output[1].permute(2, 1, 0)*0.5)
        
        if self.ct != "LSTM":
            return output,hidden
        else:
            return output,hidden,cell

    def initHidden(self):
        if self.bi==False:
            return torch.zeros(self.nl, self.bs, self.hs, device=device)
        else:
            return torch.zeros(2*self.nl, self.bs, self.hs, device=device)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, params, output_size):
        super(Decoder, self).__init__()
        self.hs = params["hs"]
        self.dp = nn.Dropout(params["dd"])
        self.nl = params["nl"]
        self.bs = params["bs"]
        self.es = params["es"]
        self.em = nn.Embedding(output_size, params["es"])
        self.ct = params["ct"]
        
        
        if(params["ct"] == "LSTM"):
            self.lstm = nn.LSTM(params["es"], params["hs"], params["nl"], dropout = params["dd"])
        
        if(params["ct"] == "RNN"):
            self.rnn = nn.RNN(params["es"], params["hs"], params["nl"], dropout = params["dd"])
        
        if(params["ct"] == "GRU"):
            self.gru = nn.GRU(params["es"], params["hs"], params["nl"], dropout = params["dd"])
        
                
        self.softmax = nn.LogSoftmax(dim=1)
        
        self.out = nn.Linear(params["hs"], output_size)
        

    def forward(self, input, hidden):
        
        output = torch.relu(self.dp(self.em(input).view(-1, self.bs, self.es)))
        
        if(self.ct == "RNN"):
            output, hidden = self.rnn(output, hidden)
        
        if(self.ct == "LSTM"):
            output, (hidden, cell) = self.lstm(output, (hidden[0], hidden[1]))
        
        if(self.ct == "GRU"):
            output, hidden = self.gru(output, hidden)
            
        if self.ct == "LSTM":
            return self.softmax(self.out(output[0])), hidden, cell
        
        return self.softmax(self.out(output[0])), hidden

    def initHidden(self):
        return torch.zeros(self.nl, self.bs, self.hs, device=device)

In [ ]:
class DecoderAttention(nn.Module):
    def __init__(self, params, out_size):
        super(DecoderAttention, self).__init__()
        self.hs = params["hs"]
        self.out_size = out_size
        self.dp = params["dd"]
        self.nl = params["nl"]
        self.bs = params["bs"]
        self.es = params["es"]
        self.em = nn.Embedding(out_size, self.es)
        self.ct = params["ct"]
    
        self.attn = nn.Linear(self.hs + self.es, max_inp_len)

        self.attn_combine = nn.Linear(self.hs + self.es, self.hs)
        
        self.dpout = nn.Dropout(self.dp)
        
        if(self.ct == "GRU"):
            self.gru = nn.GRU(self.hs, self.hs, self.nl, dropout = self.dp)
            
        if(self.ct == "LSTM"):
            self.lstm = nn.LSTM(self.hs, self.hs, self.nl, dropout = self.dp)
            
        if(self.ct == "RNN"):
            self.rnn = nn.RNN(self.hs, self.hs, self.nl, dropout = self.dp)
            
        self.out = nn.Linear(self.hs, self.out_size)
        
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, enc_outputs):
        input = input.unsqueeze(0)
        output = self.dpout(self.em(input).view(-1, self.bs, self.es))
        
        if self.ct == "LSTM":
            attn_weights = F.softmax(self.attn(torch.cat((output[0], hidden[0][0]), 1)), dim=1)
        else:
            attn_weights = F.softmax(self.attn(torch.cat((output[0], hidden[0]), 1)), dim=1)
            
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),enc_outputs.permute(1, 0, 2))
        attn_applied = attn_applied.squeeze(1)
        output = torch.cat((output[0], attn_applied), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        
        if(self.ct == "GRU"):
            output, hidden = self.gru(output, hidden)
            
        if(self.ct == "LSTM"):
            output, (hidden, cell) = self.lstm(output, (hidden[0], hidden[1]))
            
        if(self.ct == "RNN"):
            output, hidden = self.rnn(output, hidden)
            
        if self.ct == "LSTM":
            return self.out(output[0]), hidden, cell, attn_weights
        
        return self.out(output[0]), hidden, attn_weights

In [ ]:
class Combine:
    def __init__(self,encoder, decoder, epochs, learning_rate, batch_size, embedding_size, cell_type, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi):
        self.teacher_forcing_ratio = 0.5
        self.runEpochs(encoder, decoder, epochs, learning_rate, batch_size, embedding_size, cell_type, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi)
        
    def findAccuracy(self, input, actual_output, cell_type, n, batch_size, encoder1, decoder1,hin):
        correct_output=0
        no_of_batch=len(input) 
        
        for i in range(no_of_batch):
            output_word = self.evaluate(input[i], cell_type, batch_size, encoder1, decoder1,hin)
            compare_output=actual_output[i]
            for j in range(len(output_word)):
                if(output_word[j]==compare_output[j]):
                    correct_output+=1
                
        return correct_output/n*100
    

    def train(self, train_para): 
        enc_hidden = train_para["enc"].initHidden()
        train_para["optim_enc"].zero_grad()
        train_para["optim_dec"].zero_grad()

        input_length = train_para["input"].size(0)
        target_length = train_para["target"].size(0)

        loss = 0
        
        if train_para["ct"] != "LSTM":
            enc_hidden = train_para["enc"](train_para["input"], enc_hidden)
            
        else:
            enc_hidden, enc_cell = train_para["enc"](train_para["input"], enc_hidden)

        dec_input = torch.tensor([SOW_token]*train_para["bs"], device=device)

        dec_hidden = enc_hidden
        
        if train_para["ct"] == "LSTM":
            dec_cell = enc_cell

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False

        if use_teacher_forcing:

            for di in range(target_length):
                if train_para["ct"] == "LSTM":
                    dec_output, dec_hidden, dec_cell = train_para["dec"](dec_input, (dec_hidden, dec_cell))
                else:
                    dec_output, dec_hidden = train_para["dec"](dec_input, dec_hidden)
                loss += train_para["criteria"](dec_output, train_para["target"][di])
                dec_input = train_para["target"][di]  # Teacher forcing

        else:
            for di in range(target_length):
                if train_para["ct"] == "LSTM":
                    dec_output, dec_hidden, dec_cell = train_para["dec"](dec_input, (dec_hidden, dec_cell))
                else:
                    dec_output, dec_hidden = train_para["dec"](dec_input, dec_hidden)
                _, topi = dec_output.topk(1)
                dec_input = topi.squeeze().detach()  # detach from history as input

                loss += train_para["criteria"](dec_output, train_para["target"][di])

        loss.backward()
        train_para["optim_enc"].step()
        train_para["optim_dec"].step()

        return loss.item() / target_length
    
    def runEpochs(self, enc, dec, ep, alpha, bs, es, ct, batch_input, batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi):
        criteria = nn.CrossEntropyLoss()
        optim_enc = optim.NAdam(enc.parameters(), lr=alpha, weight_decay = 0.0005)
        optim_dec = optim.NAdam(dec.parameters(), lr=alpha, weight_decay = 0.0005)
        
        loss = 0 

        for j in range(ep):
            print("Epoch ", j ," started ", datetime.now())
            for i in range(len(batch_input)):
                
                train_para={"input":batch_input[i],
                       "target":batch_target[i],
                       "enc":enc,
                       "dec":dec,
                       "optim_enc":optim_enc,
                       "optim_dec":optim_dec,
                       "criteria":criteria,
                       "bs":bs,
                       "ct":ct}
                l = self.train(train_para)
                loss = loss+(l*bs)

            loss_avg = loss / len(train_input)
            loss = 0
            print("Average loss ", j+1, "epochs is ", loss_avg)

            valid_accuracy = self.findAccuracy(valid_batch_input, valid_batch_output, ct, len(valid_input), bs,enc, dec,hin)
            print("Valid accuracy is ", valid_accuracy)
            wandb.log({"validation_accuracy": valid_accuracy, "training_loss": loss_avg, 'epoch': j})
        train_accuracy = self.findAccuracy(batch_input, train_actual_output, ct,len(batch_input)*bs, bs,enc,dec,hin)
        print("Train accuracy is ", train_accuracy)
        run_name = "embS_{}_nlEnc_{}_nlDec_{}_hl_{}_cellType_{}_biDir_{}_dropEnc_{}_dropDec_{}_ep_{}_bs_{}".format(es, nle, nld, hs, ct, bi, de, dd, ep, bs)
        wandb.log({"training_accuracy": train_accuracy})
        wandb.run.name = run_name
        wandb.run.save()
        wandb.run.finish()
            
            
    def evaluate(self, inp, ct, bs, enc, dec,hin):
        with torch.no_grad():
            enc_hidden = enc.initHidden()
            inp_len = inp.size(0)
            
            if ct == "LSTM":
                enc_hidden, enc_cell = enc(inp, enc_hidden)
            else:
                enc_hidden = enc(inp, enc_hidden)

            dec_input = torch.tensor([SOW_token]*bs, device=device)  
            dec_hidden = enc_hidden
            if ct == "LSTM":
                dec_cell = enc_cell
                
            output_words = [""]*bs

            for j in range(inp_len):

                if ct == "LSTM":
                    dec_output, dec_hidden, dec_cell = dec(dec_input, (dec_hidden, dec_cell))
                else:
                    dec_output, dec_hidden = dec(dec_input, dec_hidden)
                _, topi = dec_output.data.topk(1)
                for i in range(bs):
                    if topi[i].item() == EOW_token or topi[i] == 0 or topi[i].item()==PAD_token:
                        continue
                    else:
                        output_words[i] += hin.char[topi[i].item()]

                dec_input = topi.squeeze().detach()

            return output_words

In [ ]:
class CombineAttention:
    def __init__(self,encoder, decoder, epochs, learning_rate, batch_size, embedding_size, cell_type, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi):
        self.teacher_forcing_ratio = 0.5
        self.runEpochs(encoder, decoder, epochs, learning_rate, batch_size, embedding_size, cell_type, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi)
        
    def findAccuracy(self, input, actual_output, cell_type, n, batch_size, encoder1, decoder1,hin):
        correct_output=0
        no_of_batch=len(input) 
        
        for i in range(no_of_batch):
            output_word = self.evaluate(input[i], cell_type, batch_size, encoder1, decoder1,hin)
            compare_output=actual_output[i]
            for j in range(len(output_word)):
                if(output_word[j]==compare_output[j]):
                    correct_output+=1
                
        return correct_output/n*100
    

    def train(self, train_para): 
        enc_hidden = train_para["enc"].initHidden()
        train_para["optim_enc"].zero_grad()
        train_para["optim_dec"].zero_grad()

        input_length = train_para["input"].size(0)
        target_length = train_para["target"].size(0)

        loss = 0
        
        if train_para["ct"] != "LSTM":
            enc_output,enc_hidden = train_para["enc"](train_para["input"], enc_hidden)
            
        else:
            enc_output,enc_hidden, enc_cell = train_para["enc"](train_para["input"], enc_hidden)

        dec_input = torch.tensor([SOW_token]*train_para["bs"], device=device)

        dec_hidden = enc_hidden
        
        if train_para["ct"] == "LSTM":
            dec_cell = enc_cell

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False

        if use_teacher_forcing:

            for di in range(target_length):
                if train_para["ct"] == "LSTM":
                    dec_output, dec_hidden, dec_cell,attn_weights = train_para["dec"](dec_input, (dec_hidden, dec_cell),enc_output)
                else:
                    dec_output, dec_hidden,attn_weights = train_para["dec"](dec_input, dec_hidden,enc_output)
                loss += train_para["criteria"](dec_output, train_para["target"][di])
                dec_input = train_para["target"][di]

        else:
            for di in range(target_length):
                if train_para["ct"] == "LSTM":
                    dec_output, dec_hidden, dec_cell, attn_weights = train_para["dec"](dec_input, (dec_hidden, dec_cell),enc_output)
                else:
                    dec_output, dec_hidden,attn_weights = train_para["dec"](dec_input, dec_hidden,enc_output)
                _, topi = dec_output.topk(1)

                loss += train_para["criteria"](dec_output, train_para["target"][di])

        loss.backward()
        train_para["optim_enc"].step()
        train_para["optim_dec"].step()

        return loss.item() / target_length
    
    def runEpochs(self, enc, dec, ep, alpha, bs, es, ct, batch_input, batch_target, valid_batch_input,valid_batch_output,hin,nld,nle,dd,de,train_actual_output,hs,bi):
        criteria = nn.CrossEntropyLoss()
        optim_enc = optim.NAdam(enc.parameters(), lr=alpha, weight_decay = 0.0005)
        optim_dec = optim.NAdam(dec.parameters(), lr=alpha, weight_decay = 0.0005)
        
        loss = 0 

        for j in range(ep):
            print("Epoch ", j ," started ", datetime.now())
            for i in range(len(batch_input)):
                
                train_para={"input":batch_input[i],
                       "target":batch_target[i],
                       "enc":enc,
                       "dec":dec,
                       "optim_enc":optim_enc,
                       "optim_dec":optim_dec,
                       "criteria":criteria,
                       "bs":bs,
                       "ct":ct}
                l = self.train(train_para)
                loss = loss+(l*bs)

            loss_avg = loss / len(train_input)
            loss = 0
            print("Average loss ", j+1, "epochs is ", loss_avg)

            valid_accuracy = self.findAccuracy(valid_batch_input, valid_batch_output, ct, len(valid_input), bs,enc, dec,hin)
            print("Valid accuracy is ", valid_accuracy)
            wandb.log({"validation_accuracy": valid_accuracy, "training_loss": loss_avg, 'epoch': j})
        train_accuracy = self.findAccuracy(batch_input, train_actual_output, ct,len(batch_input)*bs, bs,enc,dec,hin)
        print("Train accuracy is ", train_accuracy)
        run_name = "embS_{}_nlEnc_{}_nlDec_{}_hl_{}_cellType_{}_biDir_{}_dropEnc_{}_dropDec_{}_ep_{}_bs_{}".format(es, nle, nld, hs, ct, bi, de, dd, ep, bs)
        wandb.log({"training_accuracy": train_accuracy})
        wandb.run.name = run_name
        wandb.run.save()
        wandb.run.finish()
            
    def evaluate(self, inp, ct, bs, enc, dec,hin):
        with torch.no_grad():
            enc_hidden = enc.initHidden()
            inp_len = inp.size(0)
            
            if ct == "LSTM":
                enc_output, enc_hidden, enc_cell = enc(inp, enc_hidden)
            else:
                enc_output, enc_hidden = enc(inp, enc_hidden)

            dec_input = torch.tensor([SOW_token]*bs, device=device)  
            dec_hidden = enc_hidden
            if ct == "LSTM":
                dec_cell = enc_cell
                
            output_words = [""]*bs

            for j in range(inp_len):

                if ct == "LSTM":
                    dec_output, dec_hidden, dec_cell, attn_weights = dec(dec_input, (dec_hidden, dec_cell), enc_output)
                else:
                    dec_output, dec_hidden, attn_weights = dec(dec_input, dec_hidden,enc_output)
                _, topi = dec_output.data.topk(1)
                for i in range(bs):
                    if topi[i].item() == EOW_token or topi[i] == 0 or topi[i].item()==PAD_token:
                        continue
                    else:
                        output_words[i] += hin.char[topi[i].item()]

                dec_input = topi.squeeze().detach()

            return output_words

In [ ]:
max_inp_len,max_out_len=loadData()

In [ ]:
def runSweep():
    
    config_defaults = {
        "es": 64,
        "nle": 3,
        "nld": 3,
        "hs": 256,
        "ct": "LSTM",
        "bi": True,
        "de": 0.2,
        "dd": 0.3,
        "ep": 20,
        "bs": 128,
        "attention":False,
        }
    
    wandb.init(project = 'Assignment3', entity = 'cs22m020', config=config_defaults)
    es = wandb.config.es
    nle = wandb.config.nl
    nld = wandb.config.nl
    hs = wandb.config.hs
    bs = wandb.config.bs
    ep = wandb.config.ep
    ct = wandb.config.ct
    bi = wandb.config.bi
    de = wandb.config.de
    dd = wandb.config.dd
    attention=wandb.config.attention
    
    alpha = 0.001
    
    params={"hs":hs,
            "nl":nle,
            "de":de,
            "dd":dd,
            "bs":bs,
            "es": es,
            "ep":ep,
            "ct":ct,
            "alpha":alpha,
            "bi":bi,
            "attention":attention
            }
    
    eng=Dictionary(max_inp_len)
    hin=Dictionary(max_out_len)
    train_batch_input=eng.driver(train_input,params["bs"])
    train_batch_target=hin.driver(train_output,params["bs"])
    valid_batch_input=eng.driver(valid_input,params["bs"])
    valid_batch_output=[]

    #go over the validation output and convert to batches
    for i in range(0,len(valid_output),params["bs"]):
        if i+params["bs"]>=len(valid_output):
            break
        temp=[]
        for j in range(i,i+params["bs"]):
            temp.append(valid_output[j])
        valid_batch_output.append(temp)


    train_actual_output=[]
    for i in range(0,len(train_output),params["bs"]):
        if i+params["bs"]>=len(train_output):
            break
        temp=[]
        for j in range(i,i+params["bs"]):
            temp.append(train_output[j])
        train_actual_output.append(temp)
    
    
    if attention==False:
        encoder1 = Encoder(eng.n_chars, params).to(device)
        decoder1 = Decoder(params, hin.n_chars).to(device)
        Combine(encoder1, decoder1, ep, alpha, bs, es, ct, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin, nld,nle,dd,de,train_actual_output,hs,bi)
    else:
        encoder1 = EncoderAttention(eng.n_chars, params).to(device)
        decoder1 = DecoderAttention(params, hin.n_chars).to(device)
        CombineAttention(encoder1, decoder1, ep, alpha, bs, es, ct, train_batch_input, train_batch_target, valid_batch_input,valid_batch_output,hin, nld,nle,dd,de,train_actual_output,hs,bi)

In [ ]:
sweep_config = {
  "name": "Recurrent Neural Network - Cross Entropy Loss",
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "es": {
            "values": [512, 256, 64, 32]
        },
        "nl": {
            "values": [3, 2, 1]
        },
        "hs": {
            "values": [512, 256, 128]
        },
        "ct": {
            "values": ["RNN", "GRU", "LSTM"]
        },
        "bi": {
            "values": [False, True]
        },
        "de": {
            "values": [0.2, 0.3, 0.4]
        },
        "dd": {
            "values": [0.2, 0.3, 0.4]
        },
        "ep": {
            "values": [2, 3]
        },
        "bs": {
            "values": [256, 128, 64, 32]
        },
        "attention": {
            "values": [False,True]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, entity="cs22m020", project="Assignment3")
wandb.agent(sweep_id, runSweep, count = 2)